In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/

In [1]:
import os
print(os.getcwd())


c:\Users\user\Desktop\avocado-app


In [2]:
# ===============================
# 1. Excel verisini oku
# ===============================
df = pd.read_excel (r"C:\Users\user\Downloads\Hass Avocado Ripening Photographic Dataset\Hass Avocado Ripening Photographic Dataset\Avocado Ripening Dataset.xlsx")   # kendi dosya adını yaz
image_dir = (r"C:\Users\user\Downloads\Hass Avocado Ripening Photographic Dataset\Hass Avocado Ripening Photographic Dataset\Avocado Ripening Dataset")  # fotoğraf klasörün



In [3]:
X, y = [], []

stage_4_days = df[df["Ripening Index Classification"] == 4].groupby("Sample")["Day of Experiment"].mean().to_dict()

for idx, row in df.iterrows():
    sample_id = row["Sample"]
    current_day = int(row["Day of Experiment"])
    
    # Eğer sample 4. aşamaya gelmemişse atla
    if sample_id not in stage_4_days:
        continue
    
    day_of_stage_4 = int(stage_4_days[sample_id])
    days_left = max(0, day_of_stage_4 - current_day)


    # sadece geçerli olanları ekle
    file_name = row["File Name"] + ".jpg"
    image_path = os.path.join(image_dir, file_name)
    
    if os.path.exists(image_path):
        img = cv2.imread(image_path) #resmi okumak için numpy arraye dönüştürüyor
        img = cv2.resize(img, (224, 224)) #299 da olabilir ilk bunu deneyelim
        img = preprocess_input(img) 
        X.append(img)
        y.append(days_left)

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

print("Veri boyutları:", X.shape, y.shape)


Veri boyutları: (14140, 224, 224, 3) (14140,)


In [4]:
# ===============================
# 2. Train/Test split
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ===============================
# 3. MobileNetV2 tabanlı model
# ===============================
base_model = MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,   # son sınıflandırma katmanı yok
    weights="imagenet"   # önceden eğitilmiş ağırlıklar
)

# Taban modeli dondur (önce genel özellikleri bozma)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(1, activation="linear")  # regresyon çıktısı
])

In [5]:

# ===============================
# 4. Derleme
# ===============================
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=["mae"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:


# ===============================
# 5. Eğitim
# ===============================
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    batch_size=32
)


Epoch 1/15
354/354 ━━━━━━━━━━━━━━━━━━━━ 100s 268ms/step - loss: 11.9879 - mae: 2.4925 - val_loss: 10.3557 - val_mae: 2.3538
Epoch 2/15
354/354 ━━━━━━━━━━━━━━━━━━━━ 88s 248ms/step - loss: 10.2930 - mae: 2.2762 - val_loss: 9.4981 - val_mae: 2.1890
Epoch 3/15
354/354 ━━━━━━━━━━━━━━━━━━━━ 91s 257ms/step - loss: 9.7910 - mae: 2.2162 - val_loss: 9.2173 - val_mae: 2.1202
Epoch 4/15
354/354 ━━━━━━━━━━━━━━━━━━━━ 90s 255ms/step - loss: 9.4788 - mae: 2.1581 - val_loss: 10.6987 - val_mae: 2.1832
Epoch 5/15
354/354 ━━━━━━━━━━━━━━━━━━━━ 85s 239ms/step - loss: 9.3693 - mae: 2.1384 - val_loss: 8.9631 - val_mae: 2.0540
Epoch 6/15
354/354 ━━━━━━━━━━━━━━━━━━━━ 83s 235ms/step - loss: 9.0215 - mae: 2.0945 - val_loss: 8.7488 - val_mae: 2.0615
Epoch 7/15
354/354 ━━━━━━━━━━━━━━━━━━━━ 85s 240ms/step - loss: 8.8822 - mae: 2.0696 - val_loss: 8.6512 - val_mae: 2.0284
Epoch 8/15
354/354 ━━━━━━━━━━━━━━━━━━━━ 90s 255ms/step - loss: 8.5378 - mae: 2.0319 - val_loss: 8.7970 - val_mae: 2.0406
Epoch 9/15
354/354 ━━━━━━━━

In [7]:

# ===============================
# 6. Test Sonucu
# ===============================
loss, mae = model.evaluate(X_test, y_test)
print(f"Test seti - Ortalama Hata (MAE): {mae:.2f} gün")



89/89 ━━━━━━━━━━━━━━━━━━━━ 18s 196ms/step - loss: 8.1130 - mae: 1.9786
Test seti - Ortalama Hata (MAE): 1.98 gün


In [8]:
# ===============================
# 7. Örnek Tahmin
# ===============================
preds = model.predict(X_test[:5])
print("Gerçek değerler:", y_test[:5])
print("Tahminler:", preds.flatten())


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Gerçek değerler: [ 1.  0. 18.  5.  0.]
Tahminler: [ 3.765192    0.5151802  15.531207    4.1627407   0.19492747]


In [ ]:
# Modeli kaydederken metrics'i kaldır
model.compile(optimizer='adam', loss='mse')  # metrics ekleme
model.save(r"C:\Users\user\Desktop\avocado-app\avocado_model.h5")
